In [1]:
from utils.data_management import load_data
import config as cfg

import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import os

2021-07-24 17:45:17.098440: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


# Chargement des données

## Chargement des données

In [2]:
# On profite de la puissance de Python : Les dictionnaires
datasets_map = {
    'cifar': None,
    'mnist': None,
    'fashion': None
}

for name in datasets_map.keys():
    datasets_map[name] = load_data(name, cfg.DATA_DIRECTORY)

In [3]:
datasets_map['mnist']['train']['images'].shape

(60000, 1, 28, 28)

# Construction des datasets

In [4]:
datasets_map['cifar']['train']['images'].shape

(50000, 3, 32, 32)

In [5]:
cifar_train_dataset = tf.data.Dataset.from_tensor_slices((
    datasets_map['cifar']['train']['images'],
    datasets_map['cifar']['train']['labels']
))

2021-07-24 17:45:44.935232: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-07-24 17:45:44.979998: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-07-24 17:45:44.980381: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2080 with Max-Q Design computeCapability: 7.5
coreClock: 1.095GHz coreCount: 46 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 357.69GiB/s
2021-07-24 17:45:44.980402: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-07-24 17:45:44.995864: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2021-07-24 17:45:44.995899: I tensorflow/s

In [6]:
cifar_train_dataset

<TensorSliceDataset shapes: ((3, 32, 32), ()), types: (tf.float32, tf.int64)>

# Auto-encodeur

Nous commençons par un auto-encodeur en TensorFlow à l'aide d'une classe personnalisée.

In [ ]:
# Notre bloc encodeur
class Encoder(layers.Layer):
    def __init__(self, latent_size=8, *args, **kwargs):
        super(Encoder, self).__init__(*args, **kwargs)
        
        # Couches
        self.conv1 = layers.Conv2D(filters=4, kernel_size=3, strides=1, padding="SAME", use_bias=True)
        self.conv2 = layers.Conv2D(filters=1, kernel_size=3, strides=1, padding="SAME", use_bias=True)
        
        # Activations
        self.relu = layers.ReLu()
        
        # Couche latente
        self.dense = layers.Dense(units=128, use_bias=True)
        self.latent = layers.Dense(units=latent_size, use_bias=True)
    
    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.dense(x)
        x = self.relu(x)
        return self.latent(x)

# Notre bloc décodeur
class Decoder(layers.Layer):
    def __init__(self, *args, **kwargs):
        super(Decoder, self).__init__(*args, **kwargs)
        
        # Couches
        self.de_conv1 = layers.Conv2DTranspose(filters=1, kernel_size=3, strides=1, padding="SAME", use_bias=True)
        self.de_conv2 = layers.Conv2DTranspose(filters=4, kernel_size=3, strides=1, padding="SAME", use_bias=True)
        
        # Activations
        self.relu = layers.ReLu()
        
        # Couche latente
        self.dense = layers.Dense(units=784, use_bias=True)
        self.de_latent = layers.Dense(units=128, use_bias=True)
    
    def call(self, inputs):
        x = self.de_latent(inputs)
        x = self.relu(x)
        x = self.dense(x)
        x = self.relu(x)
        x = self.de_conv2(x)
        x = self.relu(x)
        return self.de_conv1(x)


# Classe finale : combinaison encodeur - decodeur
class ConvolutionalAE(tf.keras.Model):
    def __init__(self, latent_size=8, *args, **kwargs):
        super(ConvolutionalAE, self).__init__(*args, **kwargs)
        self.encoder = Encoder(latent_size=latent_size)
        self.decoder = Decoder()

    def call(self, inputs):
        x = self.encoder(inputs)
        return self.decoder(x)

In [ ]:
lr = 0.01

model = ConvolutionalAE(latent_size=16)
loss = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.9)

train_loss = tf.keras.metrics.Mean(name='train_loss')

In [7]:
@tf.function
def run_step(inputs, labels=None):
    """
    Fonction qui va rouler une étape, soit le passage d'une batch d'exemples.
    De plus, le graph va se construire durant l'exécution à l'aide de tf.GradientTape().
    Les labels peuvent être de type 'None', auquel cas, la sortie sera comparée avec l'entrée.
    """
    with tf.GradientTape() as tape:
        outputs = model(inputs, training=True)
        if labels is None:
            loss = loss_object(inputs, outputs)
        else:
            loss = loss_object(labels, outputs)
    # Le gradient se calcule en dehors du contexte du graph
    gradients = tape.gradient(loss, model.trainable_variables)
    # La backpropagation
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    # Le suivi de la loss
    train_loss(loss)

Maintenant ce qu'il serait bien, ce serait de :
- Développer la fonction d'entrainement (les calculs réalisés lors d'une époque)
- Il semblerait qu'avec **tf.GradientTape()** nous avons la construction du graph avec les opérations effectuées durant le run
- Construire nos jeux de données séparément avec **tf.data.Dataset**
- Somme toute : continuer de suivre le tutoriel : https://www.tensorflow.org/tutorials/quickstart/advanced

### Graph des opération
Je confirme qu'avec **tf.GradientTape()** nous avons bien la construction du graph avec les opérations effectuées durant l'exécution. Le contexte permet d'enregistrer les opération réalisées.

Continuer à se renseigner sur le décorateur **@tf.function** : https://www.tensorflow.org/api_docs/python/tf/function.